# Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import illustris_python as il
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score
import seaborn as sns
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import os
import h5py

from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
basePath = '/home/kellerbw/illustris/TNG100-1' 
little_h = 0.6774
M_unit = 1e10/little_h

# Data Preprocessing
Group level

Has stellar mass > 0,
Has gas mass > 0,
Has total group mass of at least 1000 DM particles ( >4.5×10^9𝑀⊙),
Has total baryon fraction less than cosmological  (<0.15733),


SubHalo level

SubhaloFlag == 1,
Has stellar mass > 0,
Has gas mass > 0

## Data Loading

In [3]:
## Fields with units of mass
groupMassFields = ['Group_M_Crit200', 'GroupMass', 'GroupBHMass', 'GroupBHMdot', 'GroupMassType']
groupFields = ['GroupFirstSub', 'Group_R_Crit200', 'GroupStarMetallicity', 'GroupGasMetallicity', 
               'GroupSFR', 'GroupGasMetalFractions', 'GroupStarMetalFractions']+groupMassFields


In [4]:
group_data = il.groupcat.loadHalos(basePath, 99, groupFields)

In [5]:
subhaloMassFields = ['SubhaloMass','SubhaloMassType', 'SubhaloBHMass', 'SubhaloMassInRadType', 'SubhaloMassInHalfRadType', 'SubhaloMassInMaxRadType']
subhaloFields = ['SubhaloFlag', 'SubhaloGasMetalFractions', 'SubhaloMassInRad','SubhaloMassInHalfRad','SubhaloMassInMaxRad','SubhaloHalfmassRad', 'SubhaloSFR','SubhaloVmax', 'SubhaloGasMetallicity', 'SubhaloStarMetallicity', 'SubhaloStarMetalFractions','SubhaloBfldDisk','SubhaloVmaxRad',
                'SubhaloStarMetallicityMaxRad','SubhaloGasMetallicityMaxRad','SubhaloGasMetallicityHalfRad','SubhaloStarMetallicityHalfRad','SubhaloVelDisp','SubhaloBfldHalo','SubhaloGasMetallicitySfr','SubhaloHalfmassRadType'] + subhaloMassFields
subhaloFields.append('SubhaloBfldHalo')
subhalo_data = il.groupcat.loadSubhalos(basePath, 99, subhaloFields)

In [6]:
df_stellar_merger_history = pd.read_csv('stellar_merge_history.csv', index_col=0)

for col in df_stellar_merger_history.columns:
    subhalo_data[col] = df_stellar_merger_history[col].values
    subhaloFields.append(col)

In [7]:
subhaloFields

['SubhaloFlag',
 'SubhaloGasMetalFractions',
 'SubhaloMassInRad',
 'SubhaloMassInHalfRad',
 'SubhaloMassInMaxRad',
 'SubhaloHalfmassRad',
 'SubhaloSFR',
 'SubhaloVmax',
 'SubhaloGasMetallicity',
 'SubhaloStarMetallicity',
 'SubhaloStarMetalFractions',
 'SubhaloBfldDisk',
 'SubhaloVmaxRad',
 'SubhaloStarMetallicityMaxRad',
 'SubhaloGasMetallicityMaxRad',
 'SubhaloGasMetallicityHalfRad',
 'SubhaloStarMetallicityHalfRad',
 'SubhaloVelDisp',
 'SubhaloBfldHalo',
 'SubhaloGasMetallicitySfr',
 'SubhaloHalfmassRadType',
 'SubhaloMass',
 'SubhaloMassType',
 'SubhaloBHMass',
 'SubhaloMassInRadType',
 'SubhaloMassInHalfRadType',
 'SubhaloMassInMaxRadType',
 'SubhaloBfldHalo',
 'NumMajorMergersTotal',
 'NumMajorMergersLastGyr',
 'StellarMassExSitu']

In [8]:
group_conditions = (group_data['GroupMass']*M_unit > 7.5e9, group_data['GroupMassType'][:,4] > 0, 
                    group_data['GroupMassType'][:,0] > 0,
                    (group_data['GroupMass']-group_data['GroupMassType'][:,1])/group_data['GroupMass'] < 0.15733)
group_filt = np.logical_and.reduce(group_conditions)

In [9]:
sub_idx = group_data['GroupFirstSub']
subhalo_conditions = (subhalo_data['SubhaloFlag'][sub_idx], subhalo_data['SubhaloMassType'][:,4][sub_idx] > 0, 
                      subhalo_data['SubhaloMassType'][:,0][sub_idx] > 0, sub_idx != -1)


In [10]:
filt = np.logical_and.reduce(subhalo_conditions + group_conditions)

In [11]:
group_ID = np.where(filt)[0]

In [12]:
# Define StellarMassExSitu and SubhaloBfldHalo
StellarMassExSitu = np.array(subhalo_data['StellarMassExSitu'])
SubhaloBfldHalo = subhalo_data['SubhaloBfldHalo'][sub_idx][filt]

In [13]:
quasar_energy = [np.sum(il.snapshot.loadHalo(basePath, 99, ID, 5, fields=['BH_CumEgyInjection_QM'])) for ID in group_ID]
quasar_energy_arr = np.array(quasar_energy)
quasar_energy_arr[group_data['GroupBHMass'][filt] == 0] = 0
quasar_energy_arr = quasar_energy_arr.astype('float32')

In [14]:
wind_energy = [np.sum(il.snapshot.loadHalo(basePath, 99, ID, 5, fields=['BH_CumEgyInjection_RM'])) for ID in group_ID]
wind_energy_arr = np.array(wind_energy)
wind_energy_arr[group_data['GroupBHMass'][filt] == 0] = 0
wind_energy_arr = wind_energy_arr.astype('float32')

In [15]:
SFG = []
for ID in group_ID:
    SFR = il.snapshot.loadHalo(basePath, 99, ID, 0, fields=['StarFormationRate'])
    Mass = il.snapshot.loadHalo(basePath, 99, ID, 0, fields=['Masses'])
    SFG.append(sum(Mass[SFR > 0]))

In [26]:
from colossus.cosmology import cosmology
cosmo=cosmology.setCosmology('planck15')
import numpy as np

# MST feature in Gyr (converted from scale factor)
MST = []

for ID in group_ID:
    SFT = il.snapshot.loadHalo('/home/kellerbw/illustris/TNG100-1', 99, ID, 4, fields=["GFM_StellarFormationTime"])
    SFT = (SFT[SFT > 0])  # Stellar Formation Time in scale factor
    z = 1.0 / SFT - 1.0  # Converting scale factor to redshift
    age_of_universe_at_z = np.mean(cosmo.age(z)) # Getting the age of the universe at that redshift in Gyr
    MST.append(age_of_universe_at_z)  # Storing the mean stellar formation time in Gyr

In [27]:
# Metal Fields
metalFields = ['H', 'He', 'C', 'N', 'O', 'Ne', 'Mg', 'Si', 'Fe']

In [28]:
M_unit*np.array(StellarMassExSitu)

array([3.11888471e+12, 4.37782920e+11, 4.55426174e+11, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [29]:
2.60e-6*np.array(SubhaloBfldHalo)

array([5.3420007e-07, 6.3695160e-07, 5.7788208e-07, ..., 4.7431037e-13,
       9.0837539e-11, 1.6242873e-13], dtype=float32)

Reprocess Data into Pandas DataFrame

In [45]:
data_dict = {}
data_dict['QuasarEnergy'] = 2.938e53 * np.array(quasar_energy_arr, dtype=np.float64)
data_dict['WindEnergy'] = 2.938e53 * np.array(wind_energy_arr, dtype=np.float64)

data_dict['SFG'] = M_unit*np.array(SFG)
data_dict['MST'] = np.array(MST)
#data_dict['StellarMassExSitu'] = M_unit*np.array(StellarMassExSitu)
#data_dict['SubhaloBfldHalo'] = 2.60*np.array(SubhaloBfldHalo)

for k in groupFields:
    if k == 'GroupFirstSub':
        continue
    if k == 'GroupMassType':
        data_dict['GroupGasMass'] = M_unit*group_data['GroupMassType'][:,0][filt]
        data_dict['GroupStarMass'] = M_unit*group_data['GroupMassType'][:,4][filt]
        data_dict['GroupDMMass'] = M_unit*group_data['GroupMassType'][:,1][filt]
        continue
    # field_data = data_dict[k]
    if k == 'GroupGasMetalFractions':
        for j in range(9):
            data_dict['GroupGas'+metalFields[j]+'Fraction'] = group_data['GroupGasMetalFractions'][:,j][filt]
        continue
    if k == 'GroupStarMetalFractions':
        for j in range(9):
            data_dict['GroupStar'+metalFields[j]+'Fraction'] = group_data['GroupStarMetalFractions'][:,j][filt]
        continue
    if 'Group_M_Crit200' in group_data:
            data_dict['Group_M_Crit200'] = group_data['Group_M_Crit200'][filt] * M_unit
        
            data_dict[k] = group_data[k][filt]
    if k in groupMassFields:
            data_dict[k] *= M_unit
    
for k in subhaloFields:
    if k == 'SubhaloFlag':
        continue
    if k == 'SubhaloMassType':
        data_dict['SubhaloGasMass'] = M_unit*subhalo_data['SubhaloMassType'][:,0][sub_idx][filt]
        data_dict['SubhaloStarMass'] = M_unit*subhalo_data['SubhaloMassType'][:,4][sub_idx][filt]
        data_dict['SubhaloDMMass'] = M_unit*subhalo_data['SubhaloMassType'][:,1][sub_idx][filt]
        continue
    if k == 'SubhaloMassInRadType':
        data_dict['SubhaloGasMassInRad'] = M_unit*subhalo_data['SubhaloMassInRadType'][:,0][sub_idx][filt]
        data_dict['SubhaloStarMassInRad'] = M_unit*subhalo_data['SubhaloMassInRadType'][:,4][sub_idx][filt]
        data_dict['SubhaloDMMassInRad'] = M_unit*subhalo_data['SubhaloMassInRadType'][:,1][sub_idx][filt]
        continue
    if k == 'SubhaloMassInHalfRadType':
        data_dict['SubhaloGasMassInHalfRad'] = M_unit*subhalo_data['SubhaloMassInHalfRadType'][:,0][sub_idx][filt]
        data_dict['SubhaloStarMassInHalfRad'] = M_unit*subhalo_data['SubhaloMassInHalfRadType'][:,4][sub_idx][filt]
        data_dict['SubhaloDMMassInHalfRad'] = M_unit*subhalo_data['SubhaloMassInHalfRadType'][:,1][sub_idx][filt]
        continue
    if k == 'SubhaloMassInMaxRadType':
        data_dict['SubhaloGasMassInMaxRad'] = M_unit*subhalo_data['SubhaloMassInMaxRadType'][:,0][sub_idx][filt]
        data_dict['SubhaloStarMassInMaxRad'] = M_unit*subhalo_data['SubhaloMassInMaxRadType'][:,4][sub_idx][filt]
        data_dict['SubhaloDMMassInMaxRad'] = M_unit*subhalo_data['SubhaloMassInMaxRadType'][:,1][sub_idx][filt]
        continue  
        
    if k == 'SubhaloHalfmassRadType':
        data_dict['SubhaloStarHalfmassRad'] = little_h * subhalo_data['SubhaloHalfmassRadType'][:, 4][sub_idx][filt]
        data_dict['SubhaloGasHalfmassRad'] = little_h * subhalo_data['SubhaloHalfmassRadType'][:, 0][sub_idx][filt]
        continue

    if k == 'SubhaloGasMetalFractions':
        for j in range(9):
            data_dict['SubhaloGas'+metalFields[j]+'Fraction'] = subhalo_data['SubhaloGasMetalFractions'][:,j][sub_idx][filt]
        continue
    if k == 'SubhaloStarMetalFractions':
        for j in range(9):
            data_dict['SubhaloStar'+metalFields[j]+'Fraction'] = subhalo_data['SubhaloStarMetalFractions'][:,j][sub_idx][filt]
        continue
        
    data_dict[k] = subhalo_data[k][sub_idx][filt] 
    if k in subhaloMassFields: 
        data_dict[k] *= M_unit

        

In [46]:
'SubhaloBfldHalo' in data_dict.keys()

True

In [47]:
'SubhaloGasHalfmassRad' in data_dict.keys()

True

In [48]:
'SubhaloStarHalfmassRad' in data_dict.keys()

True

In [49]:
df100= pd.DataFrame(data=data_dict)

In [50]:
if 'SubhaloStarHalfmassRad' in df100.columns:
    print("SubhaloStarHalfmassRad exists in the DataFrame")
else:
    print("SubhaloStarHalfmassRad does not exist in the DataFrame")

SubhaloStarHalfmassRad exists in the DataFrame


In [51]:
df100.to_pickle("full_illustris_df100.pkl")

In [52]:
if 'SubhaloStarHalfmassRad' in df100.columns:
    print("SubhaloStarHalfmassRad exists in the DataFrame")
else:
    print("SubhaloStarHalfmassRad does not exist in the DataFrame")

SubhaloStarHalfmassRad exists in the DataFrame


In [53]:
df100['StellarMassExSitu'] = df100['StellarMassExSitu'] * M_unit
df100['SubhaloBfldHalo'] = df100['SubhaloBfldHalo']*2.60e-6
df100

,QuasarEnergy,WindEnergy,SFG,MST,Group_M_Crit200,Group_R_Crit200,GroupStarMetallicity,GroupGasMetallicity,GroupSFR,GroupGasHFraction,...,SubhaloDMMassInRad,SubhaloGasMassInHalfRad,SubhaloStarMassInHalfRad,SubhaloDMMassInHalfRad,SubhaloGasMassInMaxRad,SubhaloStarMassInMaxRad,SubhaloDMMassInMaxRad,NumMajorMergersTotal,NumMajorMergersLastGyr,StellarMassExSitu
0,1.579137e+63,3.469040e+62,1.823586e+11,4.691165,3.768139e+14,1031.668335,2.123242e-02,0.003643,98.865845,0.752393,...,1.926797e+13,1.997506e+11,1.852791e+12,6.135503e+12,1.284671e+13,3.267161e+12,9.623521e+13,7,0,3.118885e+12
1,1.173972e+63,2.778239e+62,5.362725e+10,4.445493,3.811470e+14,1035.622925,2.001553e-02,0.003415,29.315182,0.752889,...,2.908561e+13,3.672018e+11,1.142720e+12,1.087089e+13,4.238251e+13,2.155775e+12,2.587634e+14,8,0,2.024769e+12
2,1.136160e+63,2.875622e+62,2.575547e+10,4.114802,3.375381e+14,994.475647,2.047989e-02,0.003615,14.185336,0.752444,...,2.324477e+13,2.586450e+11,1.416896e+12,8.327773e+12,2.951265e+13,2.638756e+12,1.736305e+14,28,1,2.420923e+12
3,1.029329e+63,2.684330e+62,2.603744e+11,4.930408,1.710047e+14,792.820251,1.950444e-02,0.003436,277.531647,0.752881,...,4.942187e+12,1.240761e+11,7.123157e+11,1.618808e+12,9.715474e+12,1.294430e+12,6.056198e+13,6,0,8.659043e+11
4,6.654058e+62,1.642386e+62,7.998874e+10,4.505638,2.537628e+14,904.299683,1.927421e-02,0.003185,42.432865,0.753395,...,8.460772e+12,5.678038e+10,6.011619e+11,3.075935e+12,2.582030e+13,1.134208e+12,1.294521e+14,2,0,7.276360e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107862,0.000000e+00,0.000000e+00,0.000000e+00,0.943985,6.900121e+09,27.192699,8.319106e-10,0.000494,0.000000,0.758950,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.306488e+06,2.679379e+09,0,0,0.000000e+00
107863,0.000000e+00,0.000000e+00,0.000000e+00,0.981605,7.181749e+09,27.558706,9.696518e-10,0.000363,0.000000,0.759252,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.342221e+06,1.813619e+09,0,0,0.000000e+00
107864,0.000000e+00,0.000000e+00,0.000000e+00,1.787605,6.734554e+09,26.974619,9.580007e-05,0.000347,0.000000,0.759364,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.831902e+06,2.365915e+09,0,0,0.000000e+00
107865,0.000000e+00,0.000000e+00,0.000000e+00,1.327091,6.814103e+09,27.078503,9.797111e-10,0.000242,0.000000,0.759550,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.582972e+08,0,0,0.000000e+00


In [54]:
little_h = 0.6774
M_unit = 1e10 / little_h  # Mass unit conversion factor

# Listed the features that need conversion
mass_features = ['SubhaloMass', 'SubhaloMassInRad', 'SubhaloMassInHalfRad', 'SubhaloMassInMaxRad']
radius_features = ['SubhaloHalfmassRad', 'SubhaloVmaxRad']

# Applying unit conversions for mass features
for feature in mass_features:
    if feature in df100.columns:
        df100[feature] = df100[feature] * M_unit

# Applying unit conversions for radius features
for feature in radius_features:
    if feature in df100.columns:
        df100[feature] = df100[feature] * little_h

# Adding SubhaloBfldDisk feature and applying its unit conversion (multiply by 2.60)
if 'SubhaloBfldDisk' in df100.columns:
    df100['SubhaloBfldDisk'] = df100['SubhaloBfldDisk'] * 2.60


In [55]:
df100.to_pickle("full_illustris_df100.pkl")

In [56]:
df100=pd.read_pickle("full_illustris_df100.pkl")

In [57]:
df100

,QuasarEnergy,WindEnergy,SFG,MST,Group_M_Crit200,Group_R_Crit200,GroupStarMetallicity,GroupGasMetallicity,GroupSFR,GroupGasHFraction,...,SubhaloDMMassInRad,SubhaloGasMassInHalfRad,SubhaloStarMassInHalfRad,SubhaloDMMassInHalfRad,SubhaloGasMassInMaxRad,SubhaloStarMassInMaxRad,SubhaloDMMassInMaxRad,NumMajorMergersTotal,NumMajorMergersLastGyr,StellarMassExSitu
0,1.579137e+63,3.469040e+62,1.823586e+11,4.691165,3.768139e+14,1031.668335,2.123242e-02,0.003643,98.865845,0.752393,...,1.926797e+13,1.997506e+11,1.852791e+12,6.135503e+12,1.284671e+13,3.267161e+12,9.623521e+13,7,0,3.118885e+12
1,1.173972e+63,2.778239e+62,5.362725e+10,4.445493,3.811470e+14,1035.622925,2.001553e-02,0.003415,29.315182,0.752889,...,2.908561e+13,3.672018e+11,1.142720e+12,1.087089e+13,4.238251e+13,2.155775e+12,2.587634e+14,8,0,2.024769e+12
2,1.136160e+63,2.875622e+62,2.575547e+10,4.114802,3.375381e+14,994.475647,2.047989e-02,0.003615,14.185336,0.752444,...,2.324477e+13,2.586450e+11,1.416896e+12,8.327773e+12,2.951265e+13,2.638756e+12,1.736305e+14,28,1,2.420923e+12
3,1.029329e+63,2.684330e+62,2.603744e+11,4.930408,1.710047e+14,792.820251,1.950444e-02,0.003436,277.531647,0.752881,...,4.942187e+12,1.240761e+11,7.123157e+11,1.618808e+12,9.715474e+12,1.294430e+12,6.056198e+13,6,0,8.659043e+11
4,6.654058e+62,1.642386e+62,7.998874e+10,4.505638,2.537628e+14,904.299683,1.927421e-02,0.003185,42.432865,0.753395,...,8.460772e+12,5.678038e+10,6.011619e+11,3.075935e+12,2.582030e+13,1.134208e+12,1.294521e+14,2,0,7.276360e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107862,0.000000e+00,0.000000e+00,0.000000e+00,0.943985,6.900121e+09,27.192699,8.319106e-10,0.000494,0.000000,0.758950,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.306488e+06,2.679379e+09,0,0,0.000000e+00
107863,0.000000e+00,0.000000e+00,0.000000e+00,0.981605,7.181749e+09,27.558706,9.696518e-10,0.000363,0.000000,0.759252,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.342221e+06,1.813619e+09,0,0,0.000000e+00
107864,0.000000e+00,0.000000e+00,0.000000e+00,1.787605,6.734554e+09,26.974619,9.580007e-05,0.000347,0.000000,0.759364,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.831902e+06,2.365915e+09,0,0,0.000000e+00
107865,0.000000e+00,0.000000e+00,0.000000e+00,1.327091,6.814103e+09,27.078503,9.797111e-10,0.000242,0.000000,0.759550,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.582972e+08,0,0,0.000000e+00


In [45]:
features = df100.drop(['SubhaloMassInRad','SubhaloMassInHalfRad','SubhaloMassInMaxRad','GroupDMMass', 'GroupGasMass',
                    'GroupStarMass', 'SubhaloGasMass', 'SubhaloDMMass','SubhaloStarMass', 'SubhaloGasMassInMaxRad', 'SubhaloDMMassInHalfRad', 'SubhaloDMMassInMaxRad',
                    'SubhaloGasMassInRad', 'SubhaloDMMassInRad','SubhaloMass', 'WindEnergy','SubhaloStarMassInMaxRad','SubhaloStarMassInRad'], axis='columns')